# Descripción del proyecto

La compañía Sweet Lift Taxi ha recopilado datos históricos sobre pedidos de taxis en los aeropuertos. Para atraer a más conductores durante las horas pico, es necesario predecir la cantidad de pedidos de taxis para la próxima hora. Se construirá un modelo para dicha predicción.

La métrica RECM en el conjunto de prueba no debe ser superior a 48.

## Instrucciones del proyecto.

1. Descarga los datos y haz el remuestreo por una hora.
2. Analiza los datos
3. Entrena diferentes modelos con diferentes hiperparámetros. La muestra de prueba debe ser el 10% del conjunto de datos inicial.4. Prueba los datos usando la muestra de prueba y proporciona una conclusión.


Los datos se almacenan en el archivo `taxi.csv`. 	
El número de pedidos está en la columna `num_orders`.

## Importacion de librerias


In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import numpy as np


## Analisis y preparacion de datos

In [83]:
# Se cargan los datos con formato de datetime para la columna conteniendo fecha y se pasa la misma como indice de tabla
data = pd.read_csv('/datasets/taxi.csv', parse_dates = [0], index_col = [0])
data.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


In [84]:

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


In [85]:
data.isna().sum()

num_orders    0
dtype: int64

In [86]:
#Remuestreo por hora
data = data.resample('1H').sum()
data.head()

,num_orders
datetime,
2018-03-01 00:00:00,124
2018-03-01 01:00:00,85
2018-03-01 02:00:00,71
2018-03-01 03:00:00,66
2018-03-01 04:00:00,43


In [87]:
#comprobacion de orden cronologico
data.index.is_monotonic

True

In [88]:
#Funcion para creacion de caracteristicas


def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

  
    data['rolling_mean'] = (
        data['num_orders'].shift().rolling(rolling_mean_size).mean()
    )





In [89]:
#Se aplica la funcion a la tabla de datos
make_features(data, 3, 10)
data

,num_orders,year,month,day,dayofweek,lag_1,lag_2,lag_3,rolling_mean
datetime,,,,,,,,,
2018-03-01 00:00:00,124,2018,3,1,3,NaN,NaN,NaN,NaN
2018-03-01 01:00:00,85,2018,3,1,3,124.0,NaN,NaN,NaN
2018-03-01 02:00:00,71,2018,3,1,3,85.0,124.0,NaN,NaN
2018-03-01 03:00:00,66,2018,3,1,3,71.0,85.0,124.0,NaN
2018-03-01 04:00:00,43,2018,3,1,3,66.0,71.0,85.0,NaN
...,...,...,...,...,...,...,...,...,...
2018-08-31 19:00:00,136,2018,8,31,4,207.0,217.0,197.0,173.3
2018-08-31 20:00:00,154,2018,8,31,4,136.0,207.0,217.0,166.6
2018-08-31 21:00:00,159,2018,8,31,4,154.0,136.0,207.0,162.9


## Entrenamiento de modelos

In [90]:
#Creacion de conjunto de entrenamiento y prueba

train, test = train_test_split(data, shuffle=False, test_size=0.1)
train = train.dropna()
features_train = train.drop(['num_orders'], axis=1)
target_train = train['num_orders']
features_test = test.drop(['num_orders'], axis=1)
target_test = test['num_orders']



In [91]:
#check split (correlatividad de fechas)

print(train.index.min(), train.index.max())
print(test.index.min(), test.index.max())

2018-03-01 10:00:00 2018-08-13 13:00:00
2018-08-13 14:00:00 2018-08-31 23:00:00


### Modelo de Linear Regression

In [92]:


model_linear = LinearRegression()
model_linear.fit(features_train, target_train)

linear_pred_train = model_linear.predict(features_train)
linear_pred_test = model_linear.predict(features_test)

In [93]:
print('Modelo de Regresion Lineal')
#print('EAM para el conjunto de entrenamiento:', mean_absolute_error(target_train, linear_pred_train))
print('EAM para el conjunto de prueba:', mean_absolute_error(target_test, linear_pred_test))

Modelo de Regresion Lineal
EAM para el conjunto de prueba: 40.51476504252156


### Modelo RandomForestRegressor



In [94]:
#Entrenamiento y seleccion del mejor modelo de bosque aleatorio

best_score_random_forest = 10000000000
best_est_random_forest = 0
best_depth_random_forest = 0
for est in range(1, 12): # selecciona el rango del hiperparámetro
    for depth in range(1,12):
        model_forest = RandomForestRegressor(random_state=54321, max_depth= depth, n_estimators=est) # configura el número de árboles
        model_forest.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
        predictions_forest = model_forest.predict(features_test) 
        
        eam =  mean_absolute_error(target_test, predictions_forest)# calcula la puntuación de rsme en el conjunto de validación
        if eam < best_score_random_forest:
            best_model_forest = model_forest
            best_score_random_forest =  eam# guarda la mejor puntuación RSME en el conjunto de validación
            best_est_random_forest = est# guarda el número de estimadores que corresponden a la mejor puntuación de RSME
            best_depth_random_forest = depth # guarda la la profundidad que corresponde a la mejor puntacion de RSME

print("Mejor modelo 'Bosque Aleatorio':")
print("EAM:", best_score_random_forest, "", "Profundidad:",  best_depth_random_forest, "n_estimators:", best_est_random_forest)




Mejor modelo 'Bosque Aleatorio':
EAM: 40.710137601118355  Profundidad: 5 n_estimators: 3


### Modelo de LGVM

In [95]:
#Entrenamiento del modelo LGVM

best_eam_lgb = 10000000000
best_est_lgb = 0
best_num_leaves_lgb = 0
best_learning_rate_lgb = 0

for est in range(2, 5): # selecciona el rango del hiperparámetro
    for leaves in range(32, 35):
        for rate in np.linspace(0.1,0.5,3):
            clf = lgb.LGBMClassifier(learning_rate=rate, num_leaves = leaves, n_estimators = est)
            clf.fit(features_train, target_train)
            predictions_lgb =clf.predict(features_test)
            eam_lgb = mean_absolute_error(target_test, predictions_lgb)

            if eam_lgb < best_eam_lgb:
                best_clft = clf
                best_eam_lgb  =  eam_lgb # guarda la mejor puntuación RSME en el conjunto de validación
                best_est_lgb = est# guarda el número de estimadores que corresponden a la mejor puntuación de RSME
                best_num_leaves_lgb = leaves # guarda la la profundidad que corresponde a la mejor puntacion de RSME
                best_learning_rate_lgb = rate


print("Mejor modelo 'LGVM':")
print("EAM:", best_eam_lgb, "", "Leaves:",  best_num_leaves_lgb, "n_estimators:", best_est_lgb, "", 'Learning rate', best_learning_rate_lgb)



Mejor modelo 'LGVM':
EAM: 46.57692307692308  Leaves: 33 n_estimators: 3  Learning rate 0.30000000000000004


- El modelo de regresion lineal y bosque aleatorio cuentan con valores similares de EAM, y menores al del modelo de LGVM. Dado que el modelo de regresion lineal es mas sencillo, se utilizara este ultimo para las predicciones


## Prueba de cordura

In [96]:
#prediccion con el anterior

pred_previous = target_test.shift()
pred_previous.iloc[0] = target_train.iloc[-1] #Considera que el último valor de los datos de entrenamiento se puede utilizar como el primer valor de los datos de prueba
print('Prediccion con valor anterior', 'EAM:', mean_absolute_error(target_test, pred_previous))
      
    




Prediccion con valor anterior EAM: 44.97285067873303


- El valore se error EAM obtenido para un modelo que predice con el valor anterior es mayor al obtenido con el medelo de regresion lineal, comprobando asi la calidad del modelo seleccionado

## Conclusion

- Se crearon caracterisitcas a partir de los datos disponibles
- Se entrenaron 3 modelos de regresion, probando distintos parametros para obtener los optimos
- Se calculo el error EAM, para cada modelo conlcuyendo que : el modelo de regresion lineal y bosque aleatorio cuentan con valores similares de EAM, y menores al del modelo de LGVM. Dado que el modelo de regresion lineal es mas sencillo, se utilizara este ultimo para las predicciones
- El valor de EAM del modelo regresion lineal en el conjunto de prueba es de 40,5 
- se realizo una prueba de cordura, comaparando el EAM del modelo de regresion lineal seleccionado vs el error obtenido con un modelo sencillo que predice con el valor anterior comprobrando la calidad del modelo 
